# Содержание
* сводные таблицы
* фильтры и вычисления с помощью метода loc
* фильтрация пустых значений через isnull
* время в pandas
* строковые методы
* немного про учет форм слов

# Сводные таблицы
Прям как в экселе

In [1]:
import pandas as pd

In [2]:
ratings = pd.read_csv('~/Repositories/Python/.datasets/ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [6]:
len(ratings)

100004

### Упражнение
Какие оценки поставил пользователь №1 и в каком количестве?

Что делаем:
1. Фильтруем датафрейм ratings для userId = 1
2. Считаем для этого пользователя сколько он выставил единиц, двоек итд

In [9]:
ratings[ratings.userId == 1]['rating'].value_counts()

2.0    7
3.0    4
2.5    3
4.0    3
1.0    2
3.5    1
Name: rating, dtype: int64

Как это сделать для всех пользователей сразу:

In [10]:
ratings.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', aggfunc = 'count').head()

rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0
userId,,,,,,,,,,
1,NaN,2.0,NaN,7.0,3.0,4.0,1.0,3.0,NaN,NaN
2,NaN,2.0,NaN,4.0,NaN,36.0,NaN,23.0,NaN,11.0
3,NaN,NaN,NaN,1.0,3.0,18.0,9.0,11.0,4.0,5.0
4,NaN,5.0,NaN,5.0,NaN,23.0,NaN,52.0,NaN,119.0
5,NaN,NaN,1.0,NaN,3.0,3.0,27.0,42.0,19.0,5.0


In [11]:
ratings.pivot_table(index = 'userId', columns = 'rating', 
                    values = 'timestamp', aggfunc = 'count', fill_value = 0).head()

rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0
userId,,,,,,,,,,
1,0,2,0,7,3,4,1,3,0,0
2,0,2,0,4,0,36,0,23,0,11
3,0,0,0,1,3,18,9,11,4,5
4,0,5,0,5,0,23,0,52,0,119
5,0,0,1,0,3,3,27,42,19,5


можно итоги добавить

In [12]:
ratings.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0, 
                    margins = True).head()

rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0,All
userId,,,,,,,,,,,
1,0,2,0,7,3,4,1,3,0,0,20
2,0,2,0,4,0,36,0,23,0,11,76
3,0,0,0,1,3,18,9,11,4,5,51
4,0,5,0,5,0,23,0,52,0,119,204
5,0,0,1,0,3,3,27,42,19,5,100


In [33]:
dict_ = {'a':1,'b':2,'c':3}
dict_

{'a': 1, 'b': 2, 'c': 3}

In [34]:
{v: k for k, v in dict_.items()}

{1: 'a', 2: 'b', 3: 'c'}

### Упражнение
Какой пользователь выставил больше всех пятерок?

In [32]:
sum(ratings.pivot_table(index = 'userId', 
                        columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0, 
                        margins = True).iloc[:, 9])

30190

In [37]:
ratings.pivot_table(index = 'userId',
                    columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0, 
                    margins = True)[5.0]

userId
1          0
2         11
3          5
4        119
5          5
       ...  
668        8
669        4
670       11
671       19
All    15095
Name: 5.0, Length: 672, dtype: int64

In [40]:
sum(ratings.pivot_table(index = 'userId',
                        columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0, 
                        margins = True)[5.0])/2

15095.0

In [39]:
ratings.pivot_table(index = 'userId', 
                    columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0, 
                    margins = True)[5.0].sort_values(ascending=False).head()

userId
All    15095
564      408
232      243
242      219
547      214
Name: 5.0, dtype: int64

In [42]:
ratings.pivot_table(index = 'userId', 
                    columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0, 
                    margins = True).sort_values(5.0,ascending=False).head()

rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0,All
userId,,,,,,,,,,,
All,1101,3326,1687,7271,4449,20064,10538,28750,7723,15095,100004
564,0,152,0,187,0,414,0,707,0,408,1868
232,0,35,0,32,0,96,0,276,0,243,682
242,0,0,0,4,0,25,0,151,0,219,399
547,53,79,58,204,177,411,378,591,226,214,2391


### Фильтры и вычисления с помощью loc и iloc

In [12]:
log = pd.read_csv('/home/rain/Repositories/Python/.datasets/14./visit_log.csv', sep=';')
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex


### Но сначала упражнение
Сколько различных страниц есть в столбце url? Постройте топ-5 страниц по посещаемости.

Бонусный вариант: выведите топ-5 страниц без указания хоста https://host.ru

In [13]:
log['url'].value_counts()

url
https://host.ru/3c19b4ef7371864fa3    541
https://host.ru/108ce4b365afb7b88e    534
https://host.ru/23c64743ad842b9311    532
https://host.ru/1afc5618562365020a    529
https://host.ru/2fcb223266880ccf57    526
https://host.ru/3a80d6ca023aed7752    519
https://host.ru/da83c74e35e7806fe0    518
https://host.ru/a8ce0b447c10a1deaa    517
https://host.ru/9010367b0b48d2f1b9    514
https://host.ru/8e88d4703848cc0ec4    511
https://host.ru/f2297425e1328e9e35    505
https://host.ru/fd32c3d2de55949865    505
https://host.ru/4adc8a29b212051b04    504
https://host.ru/6407d4271d7b23cdf5    504
https://host.ru/ae8a036d7f44d0bcd9    503
https://host.ru/8880fd00cf23066541    501
https://host.ru/87010dbb42ea270213    500
https://host.ru/39fa884393666d45fc    498
https://host.ru/9f90e5c423a5f976d4    498
https://host.ru/1a0fe145099b114845    497
https://host.ru/060d9bdbcba66b8bb3    497
https://host.ru/a5dda93e70318570c0    496
https://host.ru/c8d9213a31839f9a3a    496
https://host.ru/b8b58337d272ee

In [67]:
len(log['url'].value_counts())

38

In [70]:
log.url.nunique()

38

In [54]:
log['url'][0][15:]

'/3c19b4ef7371864fa3'

In [62]:
log['url2'] = log.url.apply(lambda url: url[16:])
display(log.head())
log.url2.value_counts().head()

,timestamp,visit_id,url,region,user_id,traffic_source,url2
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,3c19b4ef7371864fa3
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,c8d9213a31839f9a3a
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,b8b58337d272ee7b15
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,b8b58337d272ee7b15
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,b8b58337d272ee7b15


3c19b4ef7371864fa3    541
108ce4b365afb7b88e    534
23c64743ad842b9311    532
1afc5618562365020a    529
2fcb223266880ccf57    526
Name: url2, dtype: int64

метод loc позволяет выбрать строки и столбцы в соответствии с условиями

двоеточие означает выбор всех значений

In [71]:
log.loc[:, ['user_id', 'region']].head()

,user_id,region
0,b1613cc09f,Russia
1,4c3ec14bee,Russia
2,a8c40697fb,Russia
3,521ac1d6a0,Russia
4,d7323c571c,Russia


iloc работает аналогично, но с указанием номера строки / столбца

In [72]:
log.iloc[:, [1, -3]].head()

,visit_id,user_id
0,e3b0c44298,b1613cc09f
1,6e340b9cff,4c3ec14bee
2,96a296d224,a8c40697fb
3,709e80c884,521ac1d6a0
4,df3f619804,d7323c571c


пример фильтра на страну

если столбцы не надо фильтровать, то второй параметр можно не указывать

In [73]:
log.loc[log.region == 'Russia'].head()

,timestamp,visit_id,url,region,user_id,traffic_source,url2
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,3c19b4ef7371864fa3
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,c8d9213a31839f9a3a
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,b8b58337d272ee7b15
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,b8b58337d272ee7b15
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,b8b58337d272ee7b15


пример вычисления нового столбца с НДС для страны

In [81]:
log.loc[log.region == 'Russia', 'VAT'] = 1.2
log.head(10)

,timestamp,visit_id,url,region,user_id,traffic_source,VAT
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,1.2
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,1.2
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,1.2
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,1.2
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,1.2
5,1549980742,8855508aad,https://host.ru/df646c3676cc259fa0,Russia,fc43898e47,yandex,1.2
6,1549980742,b0f66adc83,https://host.ru/b8b58337d272ee7b15,Russia,13fc55e781,paid,1.2
7,1549980754,837885c8f8,https://host.ru/108ce4b365afb7b88e,Russia,cb5082b6f6,direct,1.2
8,1549980760,af5570f5a1,https://host.ru/3004a8273caeef2867,China,45664f7af2,direct,NaN
9,1549980765,3e7077fd2f,https://host.ru/df646c3676cc259fa0,Russia,6f9de8c8b6,email,1.2


вариант с вычисляемым фильтром

In [82]:
log.loc[lambda row: row.region == 'Russia'].head(10)

,timestamp,visit_id,url,region,user_id,traffic_source,VAT
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,1.2
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,1.2
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,1.2
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,1.2
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,1.2
5,1549980742,8855508aad,https://host.ru/df646c3676cc259fa0,Russia,fc43898e47,yandex,1.2
6,1549980742,b0f66adc83,https://host.ru/b8b58337d272ee7b15,Russia,13fc55e781,paid,1.2
7,1549980754,837885c8f8,https://host.ru/108ce4b365afb7b88e,Russia,cb5082b6f6,direct,1.2
9,1549980765,3e7077fd2f,https://host.ru/df646c3676cc259fa0,Russia,6f9de8c8b6,email,1.2
12,1549980789,15ec7bf0b5,https://host.ru/8e88d4703848cc0ec4,Russia,6d1d81f7f8,paid,1.2


### Скорость метода loc
Посчитаем средний рейтинг разных жанров. Метод merge взят из материалов следующего занятия ¯ \ _ (ツ) _ / ¯

In [83]:
ratings = pd.read_csv('/home/rain/Repositories/Python/.datasets/14./ml-latest-small//ratings.csv')
movies = pd.read_csv('/home/rain/Repositories/Python/.datasets/14./ml-latest-small//movies.csv')
joined = ratings.merge(movies, how='left', on='movieId')
joined.head()

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama


In [84]:
joined['Adventure'] = joined.apply(lambda row: row.rating if 'Adventure' in row.genres else None, axis=1)
joined.head()

,userId,movieId,rating,timestamp,title,genres,Adventure
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama,NaN
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical,NaN
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller,NaN
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller,2.0
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama,NaN


Можно и через loc

In [85]:
joined['Adventure'] = joined.loc[joined.genres.str.contains('Adventure'), 'rating']
joined.head()

,userId,movieId,rating,timestamp,title,genres,Adventure
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama,NaN
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical,NaN
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller,NaN
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller,2.0
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama,NaN


Для замера времени сделаем эти операции для набор жанров

In [86]:
genres = ['Adventure', 'Animation', 'Children', 'Drama', 'Musical', 'Thriller']

In [87]:
%%time
joined = ratings.merge(movies, how='left', on='movieId')

for genre in genres:
    joined[genre] = joined.apply(lambda row: row.rating if genre in row.genres else None, axis=1)

CPU times: user 4.37 s, sys: 28.1 ms, total: 4.4 s
Wall time: 4.4 s


In [88]:
%%time
joined = ratings.merge(movies, how='left', on='movieId')

for genre in genres:
    joined[genre] = joined.loc[joined.genres.str.contains(genre), 'rating']

CPU times: user 200 ms, sys: 3.76 ms, total: 204 ms
Wall time: 203 ms


### Упражнение
Какие варианты источников трафика есть в столбце traffic_source?

Создайте столбец traffic_type, в котором для источников 'yandex' и 'google' будет стоять значение 'organic'. А для остальных вариантов - NaN.

In [103]:
log.traffic_source.apply(lambda x: 'organic' if x in ('yandex', 'google') else NaN,axis=1)

TypeError: <lambda>() got an unexpected keyword argument 'axis'

In [110]:
log['traffic_type'] = log.traffic_source.apply(lambda x: 'organic' if x in ('yandex', 'google') else 'NaN')

In [92]:
log.loc[:,['traffic_source']]

,traffic_source
0,yandex
1,direct
2,yandex
3,yandex
4,yandex
...,...
18933,paid
18934,paid
18935,yandex
18936,paid


In [99]:
log.loc[:,['traffic_source']].traffic_source[1]

'direct'

In [111]:
log

,timestamp,visit_id,url,region,user_id,traffic_source,traffic_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,NaN
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic
...,...,...,...,...,...,...,...
18933,1550094288,57e5ba8560,https://host.ru/c2382eb3d6afc8d0f3,Belarus,98b19810d0,paid,NaN
18934,1550094296,6f9389ec1b,https://host.ru/f1eb4601740d627ab0,Russia,32ebb20c13,paid,NaN
18935,1550094308,e8cf2eb8e6,https://host.ru/a5dda93e70318570c0,Belarus,b85baa8c73,yandex,organic
18936,1550094314,79530b9a67,https://host.ru/6fda01ec57f23abc9e,Russia,e154b06121,paid,NaN


In [113]:
#альтернатива
log.loc[log.traffic_source.isin(('yandex', 'google')), 'traffic_type_alt' ] = 'organic' 

In [114]:
log

,timestamp,visit_id,url,region,user_id,traffic_source,traffic_type,traffic_type_alt
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,NaN,NaN
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic,organic
...,...,...,...,...,...,...,...,...
18933,1550094288,57e5ba8560,https://host.ru/c2382eb3d6afc8d0f3,Belarus,98b19810d0,paid,NaN,NaN
18934,1550094296,6f9389ec1b,https://host.ru/f1eb4601740d627ab0,Russia,32ebb20c13,paid,NaN,NaN
18935,1550094308,e8cf2eb8e6,https://host.ru/a5dda93e70318570c0,Belarus,b85baa8c73,yandex,organic,organic
18936,1550094314,79530b9a67,https://host.ru/6fda01ec57f23abc9e,Russia,e154b06121,paid,NaN,NaN


### Методы isnull, isna
Определение пустых или None значений. По сути одинаковые методы

In [59]:
import numpy as np

In [119]:
df = pd.DataFrame({'value': [123, None, np.nan, np.NaN, np.NAN, 456]})
df

,value
0,123.0
1,NaN
2,NaN
3,NaN
4,NaN
5,456.0


### Необходимость NaN должно быть 

In [120]:
#правильно
df.value.mean()

289.5

In [121]:
#неправильно
df.fillna(0).value.mean()

96.5

In [117]:
# фильтр на пустые значения в столбце value

df.loc[pd.isnull(df.value), :]

,value
1,NaN
2,NaN
3,NaN
4,NaN


### Упражнение
Для пустых значений в столбце traffic_type выставьте значение 'other'

In [123]:
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source,traffic_type,traffic_type_alt
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,NaN,NaN
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic,organic


In [126]:
log.loc[pd.isnull(log.traffic_type), :]

,timestamp,visit_id,url,region,user_id,traffic_source,traffic_type,traffic_type_alt


In [125]:
log.loc[pd.isnull(log.traffic_type_alt), :]

,timestamp,visit_id,url,region,user_id,traffic_source,traffic_type,traffic_type_alt
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,NaN,NaN
6,1549980742,b0f66adc83,https://host.ru/b8b58337d272ee7b15,Russia,13fc55e781,paid,NaN,NaN
7,1549980754,837885c8f8,https://host.ru/108ce4b365afb7b88e,Russia,cb5082b6f6,direct,NaN,NaN
8,1549980760,af5570f5a1,https://host.ru/3004a8273caeef2867,China,45664f7af2,direct,NaN,NaN
9,1549980765,3e7077fd2f,https://host.ru/df646c3676cc259fa0,Russia,6f9de8c8b6,email,NaN,NaN
...,...,...,...,...,...,...,...,...
18931,1550094269,a7e2dfd759,https://host.ru/ef57543002ce2a0f2a,Belarus,04fec64cda,direct,NaN,NaN
18933,1550094288,57e5ba8560,https://host.ru/c2382eb3d6afc8d0f3,Belarus,98b19810d0,paid,NaN,NaN
18934,1550094296,6f9389ec1b,https://host.ru/f1eb4601740d627ab0,Russia,32ebb20c13,paid,NaN,NaN
18936,1550094314,79530b9a67,https://host.ru/6fda01ec57f23abc9e,Russia,e154b06121,paid,NaN,NaN


In [140]:
lg = log.loc[pd.isnull(log.traffic_type_alt), :]

In [134]:
lg

,timestamp,visit_id,url,region,user_id,traffic_source,traffic_type,traffic_type_alt
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,NaN,NaN
6,1549980742,b0f66adc83,https://host.ru/b8b58337d272ee7b15,Russia,13fc55e781,paid,NaN,NaN
7,1549980754,837885c8f8,https://host.ru/108ce4b365afb7b88e,Russia,cb5082b6f6,direct,NaN,NaN
8,1549980760,af5570f5a1,https://host.ru/3004a8273caeef2867,China,45664f7af2,direct,NaN,NaN
9,1549980765,3e7077fd2f,https://host.ru/df646c3676cc259fa0,Russia,6f9de8c8b6,email,NaN,NaN
...,...,...,...,...,...,...,...,...
18931,1550094269,a7e2dfd759,https://host.ru/ef57543002ce2a0f2a,Belarus,04fec64cda,direct,NaN,NaN
18933,1550094288,57e5ba8560,https://host.ru/c2382eb3d6afc8d0f3,Belarus,98b19810d0,paid,NaN,NaN
18934,1550094296,6f9389ec1b,https://host.ru/f1eb4601740d627ab0,Russia,32ebb20c13,paid,NaN,NaN
18936,1550094314,79530b9a67,https://host.ru/6fda01ec57f23abc9e,Russia,e154b06121,paid,NaN,NaN


In [137]:
lg.traffic_type_alt = lg.traffic_type_alt.apply(lambda _: 'other')

/tmp/ipykernel_268422/3388417661.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lg.traffic_type_alt = lg.apply(lambda _: 'other')


In [138]:
lg

,timestamp,visit_id,url,region,user_id,traffic_source,traffic_type,traffic_type_alt
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,NaN,NaN
6,1549980742,b0f66adc83,https://host.ru/b8b58337d272ee7b15,Russia,13fc55e781,paid,NaN,NaN
7,1549980754,837885c8f8,https://host.ru/108ce4b365afb7b88e,Russia,cb5082b6f6,direct,NaN,NaN
8,1549980760,af5570f5a1,https://host.ru/3004a8273caeef2867,China,45664f7af2,direct,NaN,NaN
9,1549980765,3e7077fd2f,https://host.ru/df646c3676cc259fa0,Russia,6f9de8c8b6,email,NaN,NaN
...,...,...,...,...,...,...,...,...
18931,1550094269,a7e2dfd759,https://host.ru/ef57543002ce2a0f2a,Belarus,04fec64cda,direct,NaN,NaN
18933,1550094288,57e5ba8560,https://host.ru/c2382eb3d6afc8d0f3,Belarus,98b19810d0,paid,NaN,NaN
18934,1550094296,6f9389ec1b,https://host.ru/f1eb4601740d627ab0,Russia,32ebb20c13,paid,NaN,NaN
18936,1550094314,79530b9a67,https://host.ru/6fda01ec57f23abc9e,Russia,e154b06121,paid,NaN,NaN


In [141]:
lg.traffic_type_alt.loc[:, ] = 'other'
lg

/tmp/ipykernel_268422/2796005470.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lg.traffic_type_alt.loc[:, ] = 'other'


,timestamp,visit_id,url,region,user_id,traffic_source,traffic_type,traffic_type_alt
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,NaN,other
6,1549980742,b0f66adc83,https://host.ru/b8b58337d272ee7b15,Russia,13fc55e781,paid,NaN,other
7,1549980754,837885c8f8,https://host.ru/108ce4b365afb7b88e,Russia,cb5082b6f6,direct,NaN,other
8,1549980760,af5570f5a1,https://host.ru/3004a8273caeef2867,China,45664f7af2,direct,NaN,other
9,1549980765,3e7077fd2f,https://host.ru/df646c3676cc259fa0,Russia,6f9de8c8b6,email,NaN,other
...,...,...,...,...,...,...,...,...
18931,1550094269,a7e2dfd759,https://host.ru/ef57543002ce2a0f2a,Belarus,04fec64cda,direct,NaN,other
18933,1550094288,57e5ba8560,https://host.ru/c2382eb3d6afc8d0f3,Belarus,98b19810d0,paid,NaN,other
18934,1550094296,6f9389ec1b,https://host.ru/f1eb4601740d627ab0,Russia,32ebb20c13,paid,NaN,other
18936,1550094314,79530b9a67,https://host.ru/6fda01ec57f23abc9e,Russia,e154b06121,paid,NaN,other


In [23]:
import scipy.stats
import numpy as np

In [26]:
d = pd.DataFrame(data={'a':[1],'b':[3],'c':[None]})
d

,a,b,c
0,1,3,None


### Правильно ли считается матожидание со значениями NaN

In [30]:
d = pd.DataFrame(data={'a':[1, 8,np.NaN],'b':[3,4,7],'c':[1,5,None]})
d

,a,b,c
0,1.0,3,1.0
1,8.0,4,5.0
2,NaN,7,NaN


In [33]:
d['a'].mean(), d['c'].mean()

(4.5, 3.0)

In [34]:
n = pd.DataFrame(data={'a':[1, 8,0],'b':[3,4,7],'c':[1,5,0]})
n

,a,b,c
0,1,3,1
1,8,4,5
2,0,7,0


In [36]:
n['a'].mean(), n['c'].mean()

(3.0, 2.0)

### Дата и время в pandas

In [4]:
log['date'] = pd.to_datetime(log['timestamp'], unit='s')
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source,date
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,2019-02-12 14:11:32
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,2019-02-12 14:11:44
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,2019-02-12 14:11:55
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,2019-02-12 14:12:05
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,2019-02-12 14:12:16


столбец datetime64[ns] теперь имеет тип даты

In [5]:
log.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18938 entries, 0 to 18937
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   timestamp       18938 non-null  int64         
 1   visit_id        18938 non-null  object        
 2   url             18938 non-null  object        
 3   region          18938 non-null  object        
 4   user_id         18938 non-null  object        
 5   traffic_source  18938 non-null  object        
 6   date            18938 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 1.0+ MB


получим час визита

In [6]:
log['hour'] = log.date.dt.hour
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source,date,hour
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,2019-02-12 14:11:32,14
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,2019-02-12 14:11:44,14
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,2019-02-12 14:11:55,14
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,2019-02-12 14:12:05,14
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,2019-02-12 14:12:16,14


### Упражнение
Дан датафрейм df с временем визита и соответствующей покупки. Сделайте следующее:
1. Создайте столбцы visit_dt и purchase_dt типа datetime из столбцов visit_datetime и purchase_datetime
2. Посчитайте разницу между временем визита и покупки (простым вычитанием)
3. Переведите полученный столбец с длительностью покупки в минуты:
```python
(df.purchase_dt - df.visit_dt).astype('timedelta64[m]')
```
4. Посчитайте среднее время покупки в минутах

In [55]:
df = pd.DataFrame({'visit_datetime': ['2019-11-04T00:05:13', '2019-11-04T00:06:19', '2019-11-04T01:35:14'], 
                   'purchase_datetime': ['2019-11-04T00:15:53', '2019-11-04T00:07:59', '2019-11-04T03:15:30']})
df

,visit_datetime,purchase_datetime
0,2019-11-04T00:05:13,2019-11-04T00:15:53
1,2019-11-04T00:06:19,2019-11-04T00:07:59
2,2019-11-04T01:35:14,2019-11-04T03:15:30


In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   visit_datetime     3 non-null      object
 1   purchase_datetime  3 non-null      object
dtypes: object(2)
memory usage: 176.0+ bytes


In [ ]:
visit_dt и purchase_dt

In [62]:
df.visit_datetime

0    2019-11-04T00:05:13
1    2019-11-04T00:06:19
2    2019-11-04T01:35:14
Name: visit_datetime, dtype: object

### Методы работы со строками

In [40]:
stats = pd.read_csv('keywords.csv')
stats.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [41]:
stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   keyword  100000 non-null  object
 1   shows    100000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.5+ MB


Проверка наличия подстроки в строке в питоне:

In [9]:
'охотник' in 'каждый охотник желает знать...'

True

Аналог в pandas:

In [10]:
stats[stats.keyword.str.contains('охотник')].head()

,keyword,shows
3072,сумеречные охотники 2 сезон,71965
3474,сумеречные охотники,66083
3654,белоснежка и охотник 2 фильм 2016,63154
4178,последний охотник на ведьм,57560
6127,последний охотник на ведьм фильм 2015,42213


[Документация](https://www.geeksforgeeks.org/python-pandas-series-str-contains/)

Syntax: Series.str.contains(pat, case=True, flags=0, na=nan, regex=True)

Parameter :
- pat : Character sequence or regular expression.
- case : If True, case sensitive.
- flags : Flags to pass through to the re module, e.g. re.IGNORECASE.
- na : Fill value for missing values.
- regex : If True, assumes the pat is a regular expression.

In [42]:
# поиск одного из нескольких слов

stats[stats.keyword.str.contains('охотник|фильм|2016')].head()

,keyword,shows
20,фильмы 2016,4486635
51,фильмы,2156641
54,фильмы онлайн,2305540
68,смотреть фильмы онлайн,1928484
86,порно фильмы,1458031


### Упражнение
Отфильтруйте датафрейм stats по поисковым запросам, которые содержат строку "погода в" и упоминают один из городов: Москва, Новосибирск, Краснодар.

In [50]:
stats[stats.keyword.isin(('погода в','Москва', 'Новосибирск', 'Краснодар'))]

,keyword,shows
1058,погода в,176720


In [54]:
stats[stats.keyword.isin(['Москва'])]

,keyword,shows


In [ ]:
stats.iloc[]

### replace

In [11]:
'отпуск начнется завтра'.replace('завтра', 'через месяц')

'отпуск начнется через месяц'

Аналог в pandas на запросах про сериалы:

In [12]:
serial = stats[stats.keyword.str.contains('сериалы')]
serial.head()

,keyword,shows
246,сериалы,587779
304,сериалы тут,503967
555,турецкие сериалы на русском языке,296403
881,русские сериалы,234262
890,сериалы онлайн,204812


In [13]:
serial.keyword.str.replace('сериалы', 'книги').head()

246                              книги
304                          книги тут
555    турецкие книги на русском языке
881                      русские книги
890                       книги онлайн
Name: keyword, dtype: object

### Как учитывать разное написание слов
Самое простое - методы upper и lower

In [14]:
serial.keyword.str.upper().head()

246                              СЕРИАЛЫ
304                          СЕРИАЛЫ ТУТ
555    ТУРЕЦКИЕ СЕРИАЛЫ НА РУССКОМ ЯЗЫКЕ
881                      РУССКИЕ СЕРИАЛЫ
890                       СЕРИАЛЫ ОНЛАЙН
Name: keyword, dtype: object

In [15]:
serial.keyword.str.lower().head()

246                              сериалы
304                          сериалы тут
555    турецкие сериалы на русском языке
881                      русские сериалы
890                       сериалы онлайн
Name: keyword, dtype: object

### Что делать если нужно учесть формы написания слов?

In [16]:
stats[stats.keyword.str.contains('рубл')].head()

,keyword,shows
1202,алиэкспресс на русском в рублях официальный сайт,161553
1602,курс гривны к рублю,125076
1736,али экспресс русская версия на русском в рубля...,117260
2132,доллары в рубли,114173
2172,100 долларов в рублях,97534


Почему нельзя просто оставить str.contains('рубл'):

In [ ]:
non_financial_search = 'рубленая котлетка'

### Библиотека [pymystem](https://pypi.org/project/pymystem3/)

In [18]:
!pip3 install pymystem3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 844.4 kB/s eta 0:00:00 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 2.0 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.8/198.8 kB 10.1 MB/s eta 0:00:00


In [ ]:
pymorphy2

In [19]:
from pymystem3 import Mystem #nltk for english and other для основы слова       pymorphy

In [23]:
search = 'курс гривны к рублю рубли рублях однушки'

In [24]:
m = Mystem()
lemmas = m.lemmatize(search)
lemmas

['курс',
 ' ',
 'гривна',
 ' ',
 'к',
 ' ',
 'рубль',
 ' ',
 'рубль',
 ' ',
 'рубль',
 ' ',
 'однушка',
 '\n']

In [25]:
' '.join(lemmas)

'курс   гривна   к   рубль   рубль   рубль   однушка \n'

### Домашнее задание 3
В датафрейме data создайте столбец lemmas, в котором вычислите леммы поисковых запросов из столбца keyword. Леммы должны иметь строковый тип.

In [26]:
data = pd.DataFrame({
    'keyword': ['курс гривны к рублю', 'доллары в рубли', '100 долларов в рублях', 'курс рубля'],
    'shows': [125076, 114173, 97534, 53546],
})

In [27]:
data

,keyword,shows
0,курс гривны к рублю,125076
1,доллары в рубли,114173
2,100 долларов в рублях,97534
3,курс рубля,53546


In [30]:
data.keyword[0]

'курс гривны к рублю'

In [33]:
m = Mystem()
buf = m.lemmatize(data.keyword[0])
buf

['курс', ' ', 'гривна', ' ', 'к', ' ', 'рубль', '\n']

In [34]:
''.join(buf)

'курс гривна к рубль\n'

In [36]:
data['lemmas'] = data.keyword.apply(lambda x: m.lemmatize(x)) 

In [37]:
data

,keyword,shows,lemmas
0,курс гривны к рублю,125076,"[курс, , гривна, , к, , рубль, \n]"
1,доллары в рубли,114173,"[доллар, , в, , рубль, \n]"
2,100 долларов в рублях,97534,"[100, , доллар, , в, , рубль, \n]"
3,курс рубля,53546,"[курс, , рубль, \n]"


In [38]:
data['lemmas'] = data.keyword.apply(lambda x: ''.join(m.lemmatize(x)) )

In [39]:
data

,keyword,shows,lemmas
0,курс гривны к рублю,125076,курс гривна к рубль\n
1,доллары в рубли,114173,доллар в рубль\n
2,100 долларов в рублях,97534,100 доллар в рубль\n
3,курс рубля,53546,курс рубль\n


In [35]:
data.iloc[:, 'lemmas'] = m.lemmatize(data.keyword)

AttributeError: 'Series' object has no attribute 'splitlines'

In [ ]:
data['lemmas'] = m.lemmatize(data)